# BIAFlows

## Nuclei Tracking 2D+t

### Fiji-workflow

The workflow treats the time-dimension as z-dimension and does a 3D-segmentation of the objects. The resulting object slices are then reduced to a center point.

In [1]:
from getpass import getpass 

publicKey = getpass("Please enter the public key: ")
privateKey = getpass("Please enter the private key: ")

argv = ['--cytomine_public_key', publicKey, 
 '--cytomine_host',  'biaflows.neubias.org',
 '--cytomine_private_key',  privateKey,
 '--cytomine_id_project',  '1695226', 
 '--cytomine_id_software', '11244427',
 '--ij_gauss_radius',  '3',
 '--ij_threshold',  '60',
 '--ij_open_radius', '7']

Please enter the public key: ········
Please enter the private key: ········


Import CytomineJob and Job and update the status information. Set the problem class to particle-tracking (PrtTrk).

In [2]:
import sys
from subprocess import call

from cytomine.models import Job
from neubiaswg5.helpers import NeubiasJob, prepare_data, upload_data, upload_metrics
from neubiaswg5 import CLASS_PRTTRK

jobID=-666
with NeubiasJob.from_cli(argv) as nj:
    nj.job.update(status=Job.RUNNING, progress=0, statusComment="Initialisation...")
    jobID = nj.job.id
    
problem_cls = CLASS_PRTTRK

ImportError: cannot import name '_validate_lengths'

#### Create local directories and download images

Create the local working directories in a subfolder jobID of the user's home folder.

- in: input images
- out: output images
- ground_truth: ground truth images
- tmp: temporary path

In [ ]:
in_images, gt_images, in_path, gt_path, out_path, tmp_path = prepare_data(problem_cls, nj, is_2d=False, **nj.flags)

Check the downloaded input and ground-truth images. In this case there is only one time-series as input image and one time-series as ground-truth image.

In [ ]:
print(in_path)
!ls -alh $in_path
print(gt_path)
!ls -alh $gt_path

#### Call the image analysis workflow 

In [ ]:
nj.job.update(progress=25, statusComment="Launching workflow...")
command = "/usr/bin/xvfb-run ./ImageJ-linux64 -macro macro.ijm \"input={}, output={}, gauss_rad={}, threshold={}, open_rad={}\" -batch".format(in_path, out_path, nj.parameters.ij_gauss_radius, nj.parameters.ij_threshold, nj.parameters.ij_open_radius)

return_code = call(command, shell=True, cwd="./fiji")  # waits for the subprocess to return

if return_code != 0:
    err_desc = "Failed to execute the ImageJ macro (return code: {})".format(return_code)
    nj.job.update(progress=50, statusComment=err_desc)
    raise ValueError(err_desc)
nj.job.update(progress=30, statusComment="Workflow finished...")

#### Visualize the result of the workflow

In [ ]:
macro = '\
open("'+in_path + '/' + str(in_images[0].object.id)+'.tif"); \n\
cellsStackID = getImageID(); \n\
run("Duplicate...", " "); \n\
cellsTitle = getTitle(); \n\
selectImage(cellsStackID); \n\
close(); \n\
open("'+out_path + '/' + str(in_images[0].object.id)+'.tif"); \n\
tracesStackID = getImageID(); \n\
run("Z Project...", "projection=[Max Intensity]"); \n\
run("3-3-2 RGB"); \n\
run("Maximum...", "radius=4"); \n\
run("8-bit"); \n\
tracesTitle = getTitle(); \n\
run("Merge Channels...", "c2="+tracesTitle+" c4="+cellsTitle+" create"); \n\
selectImage(tracesStackID); \n\
close(); \n\
overlayID = getImageID(); \n\
run("Capture Image"); \n\
selectImage(overlayID); \n\
close(); \n\
resultTitle=getTitle(); \n\
saveAs("jpeg", "'+tmp_path+'/'+str(in_images[0].object.id)+'"); \n\
close(); \n\
run("Quit"); \n\
'

file = open(tmp_path + "/visualize_tracks.ijm", "w") 
 
file.write(macro) 
file.close() 

print(macro)

In [ ]:
command = "/usr/bin/xvfb-run ./ImageJ-linux64 -macro "+tmp_path + "/visualize_tracks.ijm -batch"

return_code = call(command, shell=True, cwd="./fiji")  # waits for the subprocess to return

if return_code != 1:
    err_desc = "Failed to execute the ImageJ macro (return code: {})".format(return_code)
    nj.job.update(progress=50, statusComment=err_desc)
    raise ValueError(err_desc)

In [ ]:
from IPython.display import Image
Image(filename = tmp_path+'/'+str(in_images[0].object.id)+'.jpg')

#### Calculate metrics

In [ ]:
from neubiaswg5.metrics import computemetrics_batch
from cytomine.models import Property
import os

nj.job.update(progress=80, statusComment="Computing and uploading metrics...")
outfiles, reffiles = zip(*[
    (os.path.join(out_path, "{}.tif".format(image.object.id)),
     os.path.join(gt_path, "{}.tif".format(image.object.id)))
    for image in in_images
])

results = computemetrics_batch(outfiles, reffiles, "PrtTrk", tmp_path)

for key, value in results.items():
    print(key + ":" + str(value))
